In [1]:
# https://medium.com/analytics-vidhya/tutorial-on-text-classification-nlp-using-ulmfit-and-fastai-library-in-python-2f15a2aac065

In [2]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html 

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
    100% |████████████████████████████████| 576.2MB 43kB/s  eta 0:00:011
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/local/lib/python3.5/dist-packages/caffe2'
Consider using the `--user` option or check the permissions.

You are using pip version 18.0, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install fastai

You are using pip version 18.0, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!pip install dataclasses

Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/local/lib/python3.5/dist-packages/dataclasses.py'
Consider using the `--user` option or check the permissions.

You are using pip version 18.0, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [21]:
import fastai 
from fastai import * 
from fastai.text import * 
import pandas as pd 
import numpy as np 
from functools import partial 
import io 
import os

SyntaxError: invalid syntax (text.py, line 8)

In [7]:
from sklearn.datasets import fetch_20newsgroups 
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove= 
                             ('headers', 'footers', 'quotes'))
documents = dataset.data

In [8]:
documents[:10]

["Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n",
 "\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap

In [9]:
df = pd.DataFrame({'label':dataset.target, 'text':dataset.data})
df.shape

(11314, 2)

In [10]:
df.head()

,label,text
0,17,Well i'm not sure about the story nad it did s...
1,0,"\n\n\n\n\n\n\nYeah, do you expect people to re..."
2,17,Although I realize that principle is not one o...
3,11,Notwithstanding all the legitimate fuss about ...
4,10,"Well, I will have to change the scoring on my ..."


In [11]:
#df = df[df['label'].isin([1,10])] 
# df = df[df['label']<10]
# df = df.reset_index(drop = True)

In [12]:
df['label'].value_counts()

10    600
15    599
8     598
9     597
11    595
13    594
7     594
14    593
5     593
12    591
2     591
3     590
6     585
1     584
4     578
17    564
16    546
0     480
18    465
19    377
Name: label, dtype: int64

In [13]:
# data preprocessing

In [14]:
df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")

In [15]:
df.head()

,label,text
0,17,Well i m not sure about the story nad it did s...
1,0,Yeah do you expect people to read the ...
2,17,Although I realize that principle is not one o...
3,11,Notwithstanding all the legitimate fuss about ...
4,10,Well I will have to change the scoring on my ...


In [16]:
!pip install nltk

You are using pip version 18.0, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [17]:
import nltk 
nltk.download('stopwords') 
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')
# tokenization 
tokenized_doc = df['text'].apply(lambda x: x.split()) 
# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x:[item for item in x if 
                                    item not in stop_words]) 

# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)):
    t =' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 
df['text'] = detokenized_doc

[nltk_data] Downloading package stopwords to /home/grace/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
from sklearn.model_selection import train_test_split 
# split data into training and validation set 
df_trn, df_val = train_test_split(df, stratify = df['label'], 
                                  test_size = 0.4, 
                                  random_state = 12)
df_trn.shape, df_val.shape

((6788, 2), (4526, 2))

In [19]:
# Language model data 
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = 
                                  df_val, path = "") 
# Classifier model data 
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, 
                                      valid_df = df_val,  
                                      vocab=data_lm.train_ds.vocab, 
                                      bs=32)

NameError: name 'TextLMDataBunch' is not defined

In [ ]:
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103,  
                               drop_mult=0.7)

In [ ]:
# train the learner object with learning rate = 1e-2 
learn.fit_one_cycle(1, 1e-2)

In [ ]:
learn.save_encoder('ft_enc')

In [ ]:
learn = text_classifier_learner(data_clas, drop_mult=0.7) 
learn.load_encoder('ft_enc')

In [ ]:
learn.fit_one_cycle(1, 1e-2)

In [ ]:
# get predictions 
preds, targets = learn.get_preds() 
predictions = np.argmax(preds, axis = 1) 
pd.crosstab(predictions, targets)